The is how to manually add paper into the database

In [ ]:
import datetime
from scholar.database import Paper
from scholar.post_maker import make_all

Insert record into SQLite DB

In [ ]:
Paper.insert(
    category="Musicology",
    title="Examining Back-to-Back plays by gender, race, & sexual orientation",
    authors="Jan Diehm, Jada Watson",
    publication_info_summary="Jan Diehm, Jada Watson - The Pudding, 2023 - pudding.cool",
    journal="The Pudding",
    published_on=datetime.date(2023, 5, 1),
    link="https://pudding.cool/2023/05/country-radio/",
    snippet="The authors are a mix of musicologist, information school, Stats, and data journalism. They use simulation to estimate number of back-to-back songs in country music.",
).execute()

Remake markdown files

In [ ]:
make_all(overwrite=True)